In [ ]:
import os
import numpy as np
import cupy as cp
import package.myUtil as myUtil
from PIL import Image
from IPython.display import display

In [ ]:
DATA_PATH = '../data'
OBJ_NAME = "Cameraman"
H_SETTING = "gf"
# H_SETTING = "int_p-5_lmd-100_to-True"
# H_SETTING = "p-5_lmd-100_to-False"
CAP_DATE = "241114"
EXP_DATE = "241127"
n = 128
m = 255

In [ ]:
def create_D(n):
    I = cp.eye(n**2)

    Dx = I - cp.roll(I, 1, axis=1)
    Dx[n - 1 :: n, :] = 0

    Dy = I - cp.roll(I, n, axis=1)
    Dy[-n:, :] = 0

    D = cp.vstack([Dx, Dy])

    return D

In [ ]:
def gradient(f):
    """Forward difference gradient operator."""
    grad = cp.zeros_like(f)
    grad[:-1] = f[1:] - f[:-1]
    grad[-1] = 0  # Boundary condition
    return grad


def divergence(p):
    """Adjoint of the gradient operator."""
    div = cp.zeros_like(p)
    div[0] = -p[0]
    div[1:] = -(p[1:] - p[:-1])
    return div


def primal_dual(H, g, tau, max_iter=100, theta=1.0):
    n = H.shape[1]
    f = cp.zeros(n)
    f_prev = f.copy()
    p = cp.zeros(n)
    D = create_D(n)

    # Parameters
    L = 2  # Operator norm of the gradient (for 1D case)
    tau = tau
    sigma = 1.0 / (tau * L**2)

    for k in range(max_iter):
        # Extrapolation step
        f_bar = f + theta * (f - f_prev)

        # Dual variable update
        q = p + sigma * D @ f_bar
        p = q / cp.maximum(1.0, cp.abs(q) / tau)

        # Save previous f
        f_prev = f.copy()

        # Primal variable update
        grad_F = -2 * H.T @ (g - H @ f)
        f = f - tau * (grad_F + divergence(p))

        # Projection onto [0,1]^n
        f = cp.clip(f, 0, 1)

        error = cp.linalg.norm(f - f_prev)
        print(f"error: {error}")

        # (Optional) Check for convergence
        if error < 1e-5:
            break

    return f

In [ ]:
captured = Image.open(f"{DATA_PATH}/capture_{CAP_DATE}/{OBJ_NAME}.png").convert("L")
captured = cp.asarray(captured)
black = myUtil.calculate_bias(m**2, DATA_PATH, CAP_DATE)
g = captured.ravel() - black

In [ ]:
H = cp.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{H_SETTING}.npy").astype(cp.float32)
print("H shape:", H.shape, "type(H):", type(H), "H.dtype:", H.dtype)

In [ ]:
tau = 1

f_est = primal_dual(H, g, tau, max_iter=500)
f_est = cp.asnumpy(f_est.reshape(n, n))
f_image = Image.fromarray((f_est * 255).astype(np.uint8), mode="L")
display(f_image)

if not os.path.exists(f"{DATA_PATH}/{EXP_DATE}/reconst"):
    os.makedirs(f"{DATA_PATH}/{EXP_DATE}/reconst")
SAVE_PATH = f"{DATA_PATH}/{EXP_DATE}/reconst/{OBJ_NAME}_{H_SETTING}_primal_t-{tau}.png"
f_image.save(SAVE_PATH, format="PNG")
print(SAVE_PATH)